# "_Synchrotron infrared microscopy at the French Synchrotron Facility SOLEIL_" 

## P. Dumas et al , _Infrared Physics and Technology_ *49*-(1–2), 152–160 (2006).


In [ ]:
__author__ = ['Rafael Celestre']
__contact__ = 'rafael.celestre@synchrotron-soleil.fr'
__license__ = 'CECILL-2.1'
__copyright__ = 'Synchrotron SOLEIL, Saint Aubin, France'
__created__ = '2025.11.27'
__changed__ = '2025.11.28'

import sys

import barc4sr as sr
import barc4sr.plotting as pt

import numpy as np

print(sys.executable)
print(sys.version)

%load_ext autoreload
%autoreload 2

## Electron beam

In [ ]:
electron_beam = sr.ElectronBeam()
electron_beam.from_rms(energy=2.75, energy_spread=0.1025/100, current=0.500,
                       x=180E-6, xp=30.5e-6, y=8E-6, yp=4.5E-6)

electron_beam.print_rms()

## Magnetic structure

In [ ]:
mag_field_intensity = 1.7114
mag_field_length = 1.052433
mag_field_edge_length = 10e-3

extraction_angle = 1.25*np.pi/180

straight_section = 12.405

observation_point = 1324.45e-3

collection_h_ang = 78e-3
collection_v_ang = 20e-3

nx = 501

wavelength = 10e-6

In [ ]:
def _number_of_pixels(_theta, _R, _wavelength, _p=1):
    """Implementation of Eq. 5 from 10.1364/OE.27.028750"""

    return int(np.ceil(_p*_theta*_theta*_R/_wavelength))

In [ ]:
ny = int(nx*collection_v_ang/collection_h_ang)

window_h = np.tan(collection_h_ang / 2) * observation_point * 2
window_v = np.tan(collection_v_ang / 2) * observation_point * 2

In [ ]:
Nx = _number_of_pixels(collection_h_ang, observation_point, wavelength, _p = 1)
Ny = _number_of_pixels(collection_v_ang, observation_point, wavelength, _p = 1)

print('Recommended sampling:')
print(f'>> Nx: {Nx} (used: {nx})')
print(f'>> Ny: {Ny} (used: {ny})')

## Bending magnet: ```BendingMagnetSource```

In [ ]:
bm = sr.MagneticStructure(magnet_type="bm",
                          B=mag_field_intensity,
                          field_length=mag_field_length,
                          extraction_angle=extraction_angle,
                          center=None)

xray_source = sr.BendingMagnetSource(electron_beam=electron_beam,
                                     magnetic_structure=bm)
xray_source.configure(verbose=True)
extraction = (xray_source.field_length/2 - xray_source.center)

## Double bending magnet: ```multi_bm_magnetic_field```

In [ ]:
magnets = [
    {'B': -mag_field_intensity, 'length': mag_field_length,
     's0': -straight_section/2 - mag_field_length/2, 'soft_edge': {"edge_length": mag_field_edge_length}},
    {'B': -mag_field_intensity, 'length': mag_field_length,
     's0': +straight_section/2 + mag_field_length/2, 'soft_edge': {"edge_length": mag_field_edge_length}}
] 
mag_field = sr.multi_bm_magnetic_field(magnets=magnets, padding=.15, step_size=0.5e-4, verbose=True)

In [ ]:
bm = sr.MagneticStructure(magnet_type="arb", magnetic_field=mag_field)

xray_source = sr.ArbitraryMagnetSource(electron_beam=electron_beam,
                                       magnetic_structure=bm)
xray_source.configure(verbose=True, dS=-straight_section/2 - extraction)

In [ ]:
eBeamTraj = sr.electron_trajectory(light_source=xray_source, verbose=True)
pt.plot_electron_trajectory(eBeamTraj, direction='h')

In [ ]:
res = sr.trace_chief_rays(eTraj=eBeamTraj, n_rays=51, B_rel_threshold=1E-5)
pt.plot_chief_rays(res, direction='h', ray_length=1)   

In [ ]:
xray_source.configure(verbose=True, si=-13, sf=.55)
eBeamTraj = sr.electron_trajectory(light_source=xray_source, verbose=True)
pt.plot_electron_trajectory(eBeamTraj, direction='h')

In [ ]:
file_name = f'db_edge_{np.round(wavelength*1e6, 1):.1f}um_extraction_{np.round(extraction_angle*180/np.pi, 2):.2f}deg_me'.replace('.', 'p')
wavefront = sr.wavefront(light_source=xray_source, photon_energy=sr.energy_wavelength(wavelength, 'm'),
                        hor_slit=window_h, ver_slit=window_v, hor_slit_n=int(nx*3), ver_slit_n=int(ny*3), 
                        observation_point=observation_point, number_macro_electrons=1, verbose=True,
                        radiation_polarisation=['T', 'LH', 'LV'], file_name=file_name)

pt.plot_wavefront(wavefront, show_phase=False)

In [ ]:
power = sr.power_density(light_source=xray_source, hor_slit=window_h, ver_slit=window_v,
                         hor_slit_n=nx, ver_slit_n=ny,
                         observation_point=observation_point, verbose=True,
                         radiation_polarisation=['T', 'LH', 'LV'])

pt.plot_power_density(power)

# Source spectrum

In [ ]:
wavelength_array = [1e-6, 10e-6, 50e-6, 100e-6, 200e-6, 500e-6, 1e-3]
energy_array = [sr.energy_wavelength(wl, 'm') for wl in wavelength_array]

wft = []

for en in energy_array:
    print(f'Calculating for photon energy: {en:.3e} eV')
    wavefront = sr.wavefront(light_source=xray_source, photon_energy=en,
                            hor_slit=window_h, ver_slit=window_v, hor_slit_n=nx, ver_slit_n=ny, 
                            observation_point=observation_point, number_macro_electrons=1, verbose=False,
                            radiation_polarisation=['T'], file_name=None)
    
    wft.append(wavefront)


In [ ]:
flux = []
for wf in wft:
    dx = wf["axis"]["x"][1] - wf["axis"]["x"][0]
    dy = wf["axis"]["y"][1] - wf["axis"]["y"][0]
    flux.append(np.sum(wf["intensity"]['T'] * dx * 1e3 * dy * 1e3))

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.loglog(np.asarray(wavelength_array)*1E6, flux, marker='o')
plt.xlabel('wavelength (µm)')
plt.ylabel('photon flux (photons/s/0.1%bw)')
plt.ylim(1e13, 0.5E15)
plt.grid(which='both', linestyle='--', linewidth=0.5)
plt.show()